In [1]:
import pandas as pd
import numpy as np
import re
import collections
np.random.seed(1337)

In [2]:
train = pd.read_csv("data/raw/train.csv", header=None, names=['rating', 'review'])
test = pd.read_csv("data/raw/test.csv", header=None, names=['rating', 'review'])

In [3]:
my_dict = collections.defaultdict(list)
for _, row in train.iterrows():
    my_dict[row.rating].append(row.to_dict())

In [4]:
tr_r = 0.7
val_r = 0.3

In [5]:
final_list = []

for _, item_list in my_dict.items():
    np.random.shuffle(item_list)

    n_total = len(item_list)
    n_train = int(tr_r * n_total)
    n_val = int(val_r * n_total)

    for item in item_list[:n_train]:
        item['split'] = 'train'
    
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'

    final_list.extend(item_list)

In [6]:
for _, row in test.iterrows():
    row_dict = row.to_dict()
    row_dict['split'] = 'test'
    final_list.append(row_dict)

In [7]:
final_reviews = pd.DataFrame(final_list)

In [8]:
final_reviews.split.value_counts()

split
train    392000
val      168000
test      38000
Name: count, dtype: int64

In [9]:
final_reviews.split.value_counts()

split
train    392000
val      168000
test      38000
Name: count, dtype: int64

In [10]:
final_reviews.review.head()

0    The entrance was the #1 impressive thing about...
1    I'm a Mclover, and I had no problem\nwith the ...
2    Less than good here, not terrible, but I see n...
3    I don't know if I can ever bring myself to go ...
4    Food was OK/Good but the service was terrible....
Name: review, dtype: object

In [11]:
def preprocess(text):
    if type(text) == float:
        print(text)
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

final_reviews.review = final_reviews.review.apply(preprocess)
final_reviews.rating = final_reviews.rating.apply({1: "negative", 2: "positive"}.get)

In [12]:
final_reviews.head()

,rating,review,split
0,negative,the entrance was the impressive thing about th...,train
1,negative,"i m a mclover , and i had no problem nwith the...",train
2,negative,"less than good here , not terrible , but i see...",train
3,negative,i don t know if i can ever bring myself to go ...,train
4,negative,food was ok good but the service was terrible ...,train


In [13]:
final_reviews.to_csv('data/processed/03_01_data_processed.csv', index=False)